In [1]:
# ✅ SETUP

# Uninstall unnecessary packages and install Google GenAI SDK
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"



# Import the SDK
from google import genai
from google.genai import types


from IPython.display import Markdown

# Check version
genai.__version__


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


'1.7.0'

In [2]:
# ✅ API KEY SETUP

# Set up your API key stored in Kaggle Secrets
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)


In [3]:
# ✅ CHECK SUPPORTED MODELS

# Explore models that support tuning (for future custom model training)
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)



models/gemini-1.5-flash-001-tuning


In [4]:
# ✅ PROMPT: Personalized Travel Itinerary

client = genai.Client(api_key=GOOGLE_API_KEY)


prompt = """
You are a smart travel assistant specialized in creating personalized travel plans for tourists in Sri Lanka.

User Input:
- Duration: 3 days
- Preferences: Nature, Hiking, Waterfalls
- Region: Central Province (Kandy, Nuwara Eliya)

Requirements:
1. Generate a 3-day travel itinerary

2. Each day should have:
   - title
   - description
   - list of activities
   - travel tips

Return only the JSON without explanation.

Please respond with a JSON in this exact format:

{
  "tripName": "string",
  "region": "string",
  "duration": "string",
  "theme": "string",  // comma-separated
  "itinerary": [
    {
      "day": 1,
      "title": "string",
      "description": "string",
      "activities": [
        {
          "name": "string",
          "duration": "string",
          "details": "string"
        }
      ],
      "travelTips": ["string"]
    }
  ]
}

"""

high_temp_config = types.GenerateContentConfig(temperature=2.0)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=high_temp_config,
    contents=prompt)






In [5]:
import json
from IPython.display import Markdown, display

# Step 1: Clean and Parse the JSON
raw = response.text.strip()

# If it's wrapped with triple backticks, remove them
if raw.startswith("```json"):
    raw = raw.replace("```json", "").replace("```", "").strip()

# Attempt to parse the JSON response
try:
    data = json.loads(raw)
except json.JSONDecodeError as e:
    print("❌ JSON Decode Error:", e)
    print("Raw response:", raw)
    raise

# Step 2: Display Trip Overview
trip_title = data.get('tripName') or data.get('title', 'Untitled Trip')
duration = data.get('duration', 'N/A')
region = data.get('region', 'N/A')
theme = data.get('theme') or ", ".join(data.get('preferences', []))

display(Markdown(f"# 🌍 {trip_title}"))
display(Markdown(f"**Duration:** {duration}  \n**Region:** {region}  \n**Theme:** {theme}"))

# Step 3: Loop through each day and display info
days = data.get('itinerary') or data.get('days', [])
for idx, day in enumerate(days, start=1):
    day_title = day.get('title', f'Day {idx}')
    day_desc = day.get('description', '')

    display(Markdown(f"---\n## 🗓️ {day_title}"))
    display(Markdown(f"**Description:** {day_desc}"))

    # Activities
    activities = day.get('activities', [])
    if activities:
        display(Markdown("### ✅ Activities"))
        for act in activities:
            act_name = act.get('name', 'Unnamed Activity')
            act_duration = act.get('duration', 'N/A')
            act_description = act.get('description') or act.get('details', '')
            act_time = act.get('time', '')
            activity_md = f"- **{act_name}** ({act_time}, *{act_duration}*)  \n  {act_description}"
            display(Markdown(activity_md))

    # Travel Tips
    tips = day.get('travelTips') or day.get('travel_tips', [])
    if tips:
        tips_md = "\n".join([f"- {tip}" for tip in tips])
        display(Markdown("### 💡 Travel Tips"))
        display(Markdown(tips_md))


# 🌍 Central Sri Lanka Nature Escape

**Duration:** 3 days  
**Region:** Central Province  
**Theme:** Nature, Hiking, Waterfalls

---
## 🗓️ Kandy's Cultural & Natural Beauty

**Description:** Explore Kandy's highlights, blending cultural sites with scenic viewpoints and refreshing waterfalls.

### ✅ Activities

...

- **Bakers Falls** (, *1 hour*)  
  Visit Baker's falls at Horton Plains.

- **World's End** (, *30 min*)  
  Enjoy the panaromic views at the world's end at Horton Plains.

- **Return to Kandy/Departure** (, *Variable*)  
  Return to Kandy (or onward travel) in the late afternoon.

### 💡 Travel Tips

- Wear comfortable hiking shoes and layers of clothing, as the weather in Horton Plains can change quickly.
- Bring water and snacks, as there are limited facilities within the park.
- Entry fees for Horton Plains are required and can be purchased at the entrance or pre-booked through some tour operators.
- Be aware of wildlife, including sambar deer and endemic birds.